#TODO
1. splitten op jaartal voordat Xtrain en Ytrain gemaakt worden (2015 train en dan 2016 test maken)
2. je mag alleen data gebruiken die tot de XOR-split gebeurt is
3. van te voren een split kiezen (accept/reject) doctype komt niet altijd vaker voor in dezelfde trace
4. risk factor is altijd 1

In [2]:
import json
import sys
from pathlib import Path
import os
import pickle
import pandas as pd
from copy import deepcopy
from tqdm import tqdm
import json
from pathlib import Path
import numpy as np

module_path = str(Path.cwd().parents[0] / "Q1")
if module_path not in sys.path:
    sys.path.append(module_path)
import utils

module_path2 = str(Path.cwd().parents[0] / "Q4/PyDREAM/pydream")
if module_path2 not in sys.path:
    sys.path.append(module_path2)
    
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_import_factory
from pm4py.objects.petri.exporter.exporter import pnml as pnml_exporter
from pm4py.objects.petri.importer.importer import pnml as pnml_importer
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
# from pm4py.objects.log.importer.csv import importer as csv_importer
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.log.util import get_log_representation

from LogWrapper import LogWrapper
from EnhancedPN import EnhancedPN
from predictive.nap.NAP import NAP
from predictive.nap.NAPr import NAPr
from util.TimedStateSamples import loadTimedStateSamples

from pm4py.visualization.petrinet import visualizer as pn_vis_factory
from graphviz import *

# Import utils.py from Q1 folder
module_path = str(Path.cwd().parents[0] / "Q1")
if module_path not in sys.path:
    sys.path.append(module_path)
import utils

from pm4py.visualization.petrinet import visualizer
from pm4py.algo.enhancement.decision import algorithm as decision_mining

# len(log.loc[log['Undesired Outcome 1'] == True])
pd.set_option('display.max_columns', None)

/home/rowan/school/process-mining-assignment-2/../pydream
/home/rowan/school/process-mining-assignment-2/pydream
/home/rowan/school/process-mining-assignment-2/pydream


/home/rowan/.local/lib/python3.8/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Using TensorFlow backend.


In [6]:
def create_categories_doctype_subprocess(dataframe):
    """Returns dataframe which has the doctype_subprocess value filled in"""
    for idx, row in enumerate(tqdm(dataframe.values)):
        value =  dataframe.loc[idx]["doctype"] + "_" + dataframe.loc[idx]["subprocess"]
        dataframe.at[idx, "doctype_subprocess"] = value
    
    return dataframe 

def create_activity_categories(dataframe):
    start = ["income", "initialize", "begin", "create"]
    during = ["editing", "performed", "calculate", "decide", "change", "correction", "revoke", "clear", "valid", "save", "plan", "prepare", "original", "check", "offline", "insert", "remove", "pre-check"]
    end = ["finish", "abort", "discard", "withdraw", "approve", "refuse"]
    activity_type = ["Payment", "Entitlement", "Control", "Parcel", "Geo", "Reference", "Department", "Inspection"]
    parsed = []
    not_parsed = []
    mapping = dict()
    to_parse = deepcopy(dataframe["Activity"].unique())

    for idx, item in enumerate(tqdm(to_parse)):
        filtered = False

        for word in start:
            if word in item:
                for activity in activity_type:
                    if activity in item:
                        filtered = True
                        mapping[to_parse[idx]] = "Start_" + activity
                        parsed.append(to_parse[idx])
                        break

        for word in during:
            if word in item:
                for activity in activity_type:
                    if activity in item:
                        filtered = True
                        mapping[to_parse[idx]] = "During_" + activity
                        parsed.append(to_parse[idx])
                        break

        for word in end:
            if word in item:
                for activity in activity_type:
                    if activity in item:
                        filtered = True
                        mapping[to_parse[idx]] = "End_" + activity
                        parsed.append(to_parse[idx])
                        break

        if not filtered:
            not_parsed.append(to_parse[idx])

    if len(not_parsed) > 0:
        raise ValueError(f"Not all activities were correctly categorized as 'start', 'during', 'end' or were not given an 'activity_type'. \n It concerns the following items: \n \n {not_parsed}")
    
    total = len(set(val for dic in [mapping] for val in mapping.values()))
    print(f"There are {total} categories remaining.")
    
    return mapping

def add_mapping_to_df(dataframe):
    for idx, item in enumerate(tqdm(dataframe.values)):
        dataframe.at[idx, "Categorized_Activity"] = mapping[dataframe.loc[idx]["Activity"]]
        
def save_results(miner_name, filename, results):
    with open(filename, "w") as file:
        file.write(miner_name)
        file.write("\n")
        file.write("Fitness: " + str(results[0]))
        file.write("\n")
        file.write("Generalization: " + str(results[1]))
        file.write("\n")
        file.write("Simplicity: " + str(results[2]))
        file.write("\n")

def analyze_petri(log, miner_name, net, initial_marking, final_marking, filter_type="doctype"):
    start = time.perf_counter()

    # fitness based on toked-based replay
    fitness = replay_fitness_evaluator.apply(log, net, initial_marking, final_marking, 
              variant=replay_fitness_evaluator.Variants.TOKEN_BASED,
              parameters={constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Categorized_Activity"})
    
    gc.collect()

    # precision based on token-based replay
    #prec = precision_evaluator.apply(df2, net, initial_marking, final_marking, variant=precision_evaluator.Variants.ALIGN_ETCONFORMANCE, parameters={constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Categorized_Activity"})

    #print(f"[ Finished Precision metric: {time.perf_counter() - start} sec]")

    # collect garbage
    #gc.collect()

    # generalization based on token-based replay
    gen = generalization_evaluator.apply(log, net, initial_marking, final_marking, parameters={constants.PARAMETER_CONSTANT_CASEID_KEY: 'case:concept:name', constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Categorized_Activity", constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: "time:timestamp"})
                                         

    # collect garbage
    gc.collect()

    print(f"[ Finished Generalization metric: {time.perf_counter() - start} sec]")

    # simplicity measured by the inverse arc degree
    simp = simplicity_evaluator.apply(net)

    print(f"[Finished: {time.perf_counter() - start}]")
    results = [fitness, gen, simp]
    filename = miner_name + "_results.txt"
    
    # save petri net
    petrinet_name = miner_name + ".pnml"
    print(f"[Saving petrinet of {miner_name} to file :{petrinet_name}_doctype]")
    pnml_exporter.export_net(net, initial_marking, petrinet_name)
    
    # save results
    print(f"[Saving {miner_name} to file :{filename}_doctype]")
    save_results(miner_name, filename, results)

### Steps to reproduce
1. pip3 install -r requirements.txt in the PyDream folder (installation for ortools will fail)
2. pip3 install ortools==7.5.7466
3. 

One important task in customer journey optimization is    predicting the next activity in the journey. It helps for instance in preparing the business owner for the next interaction point with the application. In this part, you will start from one of the process models from Part I of your choice. 

1. Select two meaningful XOR splits in the process model you choose from Part 
    1. “meaningful” selection of the XOR depends on how you will be able to motivate it from the business perspective. You can select XOR splits with more than two outgoing arcs.

In [ ]:
df_1.head()

In [ ]:
print("Loading FILTERED parquet version of original dataframe (need to run first part of 'transform_and_filter_df.ipynb' for this). ")
df_log = utils.load_log_filtered()

In [ ]:
print("Loading FILTERED parquet version of original dataframe (need to run first part of 'transform_and_filter_df.ipynb' for this). ")
df = utils.load_parquet_filtered()

print(len(df["Case ID"].unique()))

df["Categorized_Activity"] = ""
df = df.reset_index()

# load undesired info log 
log = utils._load_pickle("../dataset/complete_undesired_outcomes_df")

# merge dataframes
df = pd.merge(df, log, how="inner", on="Case ID")

print(len(df.loc[df['Undesired Outcome 1'] == True]))

# Filter on Undesired Outcome 1
df_1 = df.loc[df['Undesired Outcome 1'] == True]

print(len(df_1["Case ID"].unique()))


# remove some columns and rename
#df_1 = df_1[['Case ID', "Activity", 'time:timestamp', "Categorized_Activity", "doctype", "concept:name"]].sort_values('time:timestamp').rename(columns={"Case ID": "case:concept:name"})
df_1 = df_1.rename(columns={"Case ID": "case:concept:name"})
# reset index for changes
df_1 = df_1.reset_index()

### Steps to reproduce
1. pip3 install -r requirements.txt in the PyDream folder (installation for ortools will fail)
2. pip3 install ortools==7.5.7466
3. 

One important task in customer journey optimization is    predicting the next activity in the journey. It helps for instance in preparing the business owner for the next interaction point with the application. In this part, you will start from one of the process models from Part I of your choice. 

1. Select two meaningful XOR splits in the process model you choose from Part 
    1. “meaningful” selection of the XOR depends on how you will be able to motivate it from the business perspective. You can select XOR splits with more than two outgoing arcs.

In [ ]:
# create the mapping
mapping = create_activity_categories(df_1)

# append column with new mapping to our old dataframe
add_mapping_to_df(df_1)

#output first 5 results
df_1.tail()

In [ ]:
len(df_1["case:concept:name"].unique())

In [ ]:
def filter_df_on_year(dataframe, year):
    """Returns filtered dataframe"""
    mask = (dataframe["time:timestamp"] < str(year)).to_numpy()
    return dataframe[mask]

def filter_df_on_end_place(dataframe, activity):
    pass

In [ ]:
len(filter_df_on_year(df_1, 2017))

In [ ]:
# csv = df_1.to_csv("df_1.csv")

# #event_stream = csv_importer.import_event_stream(os.path.join("tests", "input_data", "running-example.csv"))
log_csv = pd.read_csv('df_1.csv', sep=',')
log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
log_csv = log_csv.sort_values('time:timestamp')
event_log = log_converter.apply(log_csv)

# # automatic feature selection on the event log
# """Results in 10 features that will be one hot encoded and used to traint the model"""
# """ 8 out of 21 categories were considered important, together with index (so not timestamp) which is because
# the traces were already sorted on timestamp. Lastly the Unnamed 0 feature from the event log. 

# """
# # data, feature_names = get_log_representation.get_default_representation(event_log)
# # feature_names

In [ ]:
# generate a list of concept names
""" Results in 36 unique concept:names"""
activity_names = []
for trace in event_log:
    for event in trace:
        if event["activity"] in activity_names:
            continue
        else:
            activity_names.append(event['activity'])

print(f"Gives {len(activity_names)} cases.")

In [ ]:
"""Per case, add trace information to the dictionary consisting of:
    1. Young farmer
    2. Area
    3. Risk Factor
    4. Number of parcels
    5. Small famer
    
    Zijn al op tijd gesorteerd op time:timestamp bij het inladen van de dictionary
    
    Current Mapping:
    [0: timestamp, 
     1: activity, 
     2: identifier, 
     3: categorized_activity, 
     4: young_farmer, 
     5: area, 
     6: risk_factor, 
     7: number_parcels, 
     8: small_farmer,
     9: previous_activity,
     10: next_activity]
"""

case_dict = {}
df_1["Previous_Activity_OneHot"] = ""
df_1["Next_Activity_OneHot"] = ""
df_1["young_farmer"] = 0

for idx, row in tqdm(df_1.iterrows()):
    young_farmer = 1 if row['(case) young farmer'] == True else 0
    area = row['(case) area']
    risk_factor = row['(case) risk_factor']
    number_parcels = row['(case) number_parcels']
    small_farmer = row['(case) small farmer']
    timestamp = row['time:timestamp']
    activity = row['activity']
    identifier = row["level_0"]
    categorized_activity = row["Categorized_Activity"]
    
    data = [timestamp, activity, identifier, categorized_activity, young_farmer, 
            area, risk_factor, number_parcels, small_farmer]
    
    if row["case:concept:name"] in case_dict.keys():
        case_dict[row["case:concept:name"]].append(data)
    else:
        case_dict[row["case:concept:name"]] = []
        case_dict[row["case:concept:name"]].append(data)

In [ ]:
"""Add one-hot for previous activity"""
for sth, case in enumerate(tqdm(case_dict)):
    for idx, event in enumerate(case_dict[case]):
        if idx == 0:
            previous_activity = [0 for i in range(36)]
        else:
            previous_activity = [1 if case_dict[case][idx-1][1] == value else 0 for value in activity_names]
        event.append(previous_activity)

In [ ]:
"""Add one-hot for next activity"""
for sth, case in enumerate(tqdm(case_dict)):
    for idx, event in enumerate(case_dict[case]):
        if idx == len(case_dict[case])-1:
            next_activity = [0 for i in range(36)]
        else:
            next_activity = [1 if case_dict[case][idx+1][1] == value else 0 for value in activity_names]
        event.append(next_activity)

In [ ]:
"""Add one-hot previous, next and young-farmer binary to the dataframe"""
for case_id in tqdm(df_1["case:concept:name"].unique()):
    for idx in df_1.index[df_1["case:concept:name"] == case_id]:
        for event in case_dict[case_id]:
            if event[2] == df_1.loc[idx]["level_0"]:
                df_1.at[idx, "Previous_Activity_OneHot"] = event[-2]
                df_1.at[idx, "Next_Activity_OneHot"] = event[-1]
                df_1.at[idx, "young_farmer"] = 1 if event[4] else 0

In [ ]:
df_1.loc[df_1["case:concept:name"] == "99706b163f66b8ab"]

In [ ]:
df_2 = df_1[['Case ID', 'youngFarmer', "activity", "area", "riskFactor", "young_farmer", "Previous_Activity_OneHot"]].rename(
    columns={'(case) risk_factor': "riskFactor", '(case) area':"area", '(case) number_parcels':"parcels", 
             '(case) young farmer':"youngFarmer", 
             '(case) small farmer':"smallFarmer", "Case ID":"case:concept:name"}) # , "parcels", "activity", "Previous_Activity_OneHot", "Next_Activity_OneHot"
df_2.head()

In [ ]:
for i in df_1[:1].iterrows():
    print(i[1]["activity"])
#     if i["Previous_Activity_OneHot"][0] == [0 for i in range(36)]:
#         print(i)

In [ ]:
X = df_1[['(case) risk_factor', '(case) area', '(case) number_parcels',  'young_farmer']].to_numpy(dtype=object)
# .rename(
#     columns={'(case) risk_factor': "riskFactor", '(case) area':"area", '(case) number_parcels':"parcels", 
#              '(case) young farmer':"youngFarmer", "case:concept:name":"Case ID", 
#              '(case) small farmer':"smallFarmer"}).to_numpy() #"Previous_Activity_OneHot",
Y = df_1[["activity"]].to_numpy(dtype=object)

In [ ]:
import numpy.ma as ma
arr = range(0, 36)
newY = []
for idx, i in enumerate(Y):
    ma_arr = ma.masked_array(arr, mask=df_1["Previous_Activity_OneHot"][idx])
    if len(ma_arr.data[ma_arr.mask]) == 0:
        newY.append()
    
ma_arr = ma.masked_array(arr, mask=df_1["Previous_Activity_OneHot"][3446])

In [ ]:
ma_arr = ma.masked_array(arr, mask=df_1["Previous_Activity_OneHot"][36])
ma_arr.data[ma_arr.mask]

In [ ]:
X[0]

# Other way of adding information

In [20]:
# load filtered pandas df
df = utils.load_parquet_filtered()
df = df.rename(columns={"Case ID": "case:concept:name"})
df = df.reset_index()

"""Creates a column called doctype_subprocess with which the petri net was made"""
df = create_categories_doctype_subprocess(df)

# parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'case'}
#event_log = log_converter.apply(log_csv, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)

100%|██████████| 1784873/1784873 [10:13<00:00, 2910.02it/s]


In [23]:
# save a pickle object
df.to_pickle("./df_Q4.pkl")

df = pd.read_pickle("./df_Q4.pkl")

In [25]:
# save df as logfile
log_csv = dataframe_utils.convert_timestamp_columns_in_df(df)
log_csv = log_csv.sort_values('time:timestamp')
event_log = log_converter.apply(log_csv)

In [ ]:
# export log file TOO BIG!
# from pm4py.objects.log.exporter.xes import exporter as xes_exporter
# xes_exporter.apply(event_log, 'log_for_Q4.xes')

In [ ]:
# import log file
# log = xes_importer.apply('log_for_Q4.xes')

In [37]:
"""Calculate for each doctype_subprocess how many times it has occured before our target Split"""

import pandas as pd
import numpy as np

data = []

def parse_case(case):
    begin_payment = -1
    features = {}
    for i, e in enumerate(case):
        a = e["doctype_subprocess"]
        if a == "Payment application_Objection": # Next event is the target
            begin_payment = i
            break
        else:
            features[a] = features.get(a, 0) + 1
    features["label"] = case[begin_payment+1]["doctype_subprocess"]
    return features, case[begin_payment+1]["time:timestamp"]


for case_idx, c in enumerate(event_log):
    f = {}
#     f["year"] = c[0]["time:timestamp"].year
    f["year"] = c[0]["(case) year"]
    try:
        events, end_time = parse_case(c)
        f.update(events)
        f["duration"] = end_time - c[0]["time:timestamp"]
        f["duration"] = f["duration"].days
        data.append(f)
    except IndexError as e:
        print(events, end_time)
        print("There was no event after doctype_subprocess")
        
df = pd.DataFrame.from_dict(data)
df = df.replace(np.nan, 0)

{'Payment application_Application': 35, 'Geo parcel document_Main': 1, 'Geo parcel document_Declared': 7, 'Control summary_Main': 3, 'Reference alignment_Main': 2, 'Department control parcels_Main': 1, 'label': 'Payment application_Application'} 2016-04-27 00:00:00+00:00
There was no event after doctype_subprocess
{'Payment application_Application': 27, 'Geo parcel document_Main': 1, 'Geo parcel document_Declared': 3, 'Control summary_Main': 3, 'Reference alignment_Main': 2, 'Department control parcels_Main': 1, 'label': 'Payment application_Application'} 2016-05-11 00:00:00+00:00
There was no event after doctype_subprocess
{'Payment application_Application': 24, 'Geo parcel document_Main': 8, 'Geo parcel document_Declared': 16, 'Control summary_Main': 3, 'Reference alignment_Main': 2, 'Department control parcels_Main': 4, 'label': 'Payment application_Application'} 2016-05-13 00:00:00+00:00
There was no event after doctype_subprocess
{'Payment application_Application': 34, 'Geo parcel

In [184]:
"""Get some descriptive statistics to see if calculating this makes sense"""
df.sort_values(by="Payment application_Main").max()
df.groupby('label').count()

,year,Payment application_Application,Entitlement application_Main,Parcel document_Main,Control summary_Main,Reference alignment_Main,Department control parcels_Main,duration,Inspection_On-Site,Inspection_Remote,Payment application_Main,Entitlement application_Change,Geo parcel document_Main,Geo parcel document_Declared,Payment application_Change,Geo parcel document_Reported,Entitlement application_Objection
label,,,,,,,,,,,,,,,,,
Control summary_Main,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22
Department control parcels_Main,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
Entitlement application_Main,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777,777
Geo parcel document_Declared,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
Geo parcel document_Main,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
Geo parcel document_Reported,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
Inspection_On-Site,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Payment application_Application,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515,33515
Payment application_Objection,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71


In [ ]:
# 1. 

In [97]:
"""Generate random train-test sets based on all cases starting in 2015"""
def train_test_split(dataframe, split_percentage=0.8, random_state=42):
    train = dataframe.loc[dataframe['year'] == 2015]
    X_train = train.sample(frac = split_percentage, replace = False, random_state=200)
    X_test = train.drop(X_train.index)
    
    y_train = X_train.pop("label").to_numpy()
    y_test = X_test.pop("label").to_numpy()
    
    
    return X_train.to_numpy(), X_test.to_numpy(), y_train, y_test
    
X_train, X_test, y_train, y_test = train_test_split(df)

In [178]:
X_train[0]

array([2.015e+03, 1.300e+01, 1.900e+01, 7.000e+00, 4.000e+00, 2.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00])

## Model Training & Prediction

In [107]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [108]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1)
]

In [121]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(str(clf) + ": " + str(score))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform'): 0.8854101440200376
SVC(C=0.025, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False): 0.9073262366938009
SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False): 0.8991859737006888


In [ ]:
# ROC / verhouding Precsion/recall ipv accuracy vanwege imbalanced data

In [176]:
def predict(df, clf, idx):
    assert idx < len(df.loc[df['year'] == 2016]), "Choose a smaller index value."
    _input = df.loc[df['year'] == 2016].iloc[idx]
    real_value = _input.pop("label")
    _input = _input.to_numpy().reshape(1, -1)
    pred_val = clf.predict(_input)
    
    print(f"Predicted value: {pred_val[0]}, \n\nReal value: {real_value}")

In [177]:
predict(df, clf, 344424)

AssertionError: Choose a smaller index value.

In [ ]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [ ]:
# load the heuristics miner petrinet from Q1
net, initial_marking, final_marking = pnml_importer.import_net("../Q1/inductive_miner_undesired_outcome_2.pnml")

In [ ]:
gviz = visualizer.apply(net, initial_marking, final_marking, parameters={visualizer.Variants.WO_DECORATION.value.Parameters.DEBUG: True})
visualizer.view(gviz)

# Decision tree

In [ ]:
import pandas as pd
import numpy as np

data = []

def parse_case(case):
    begin_payment = -1
    features = {}
    for i, e in enumerate(case):
        a = e["concept:name"]
        if a == "begin payment": # Next event is the target
            begin_payment = i
            break
        else:
            features[a] = features.get(a, 0) + 1
    features["label"] = case[begin_payment+1]["concept:name"]
    return features, case[begin_payment+1]["time:timestamp"]


for case_idx, c in enumerate(desired_outcome):
    f = {}
    f["year"] = c[0]["time:timestamp"].year
    try:
        events, end_time = parse_case(c)
        f.update(events)
        f["duration"] = end_time - c[0]["time:timestamp"]
        f["duration"] = f["duration"].days
        data.append(f)
    except IndexError as e:
        print("There was no event after begin payment")
        
df = pd.DataFrame.from_dict(data)
df = df.replace(np.nan, 0)

# PyDREAM Library

In [ ]:
# Save the one-hot encoding for use in the model
def convert_ndarray_to_list(array, path='model-path/MODEL-NAME_nap_onehotdict.json'):
    lists = array.tolist()
    Path(path).touch(exist_ok=True)
    with open(path, "w") as file:
        json.dump(lists, file)
        
convert_ndarray_to_list(data)

In [ ]:
event_log[0][1]

In [ ]:
data[6]

In [ ]:
def add_next_event(self, df):
        unique_case_ids = df["Case ID"].unique()
        for case in unique_case_ids:

df.loc[df["Case ID"] == "8b99873a6136cfa6"]

In [ ]:
net, im, fm = heuristics_miner.apply(event_log, parameters={"dependency_thresh": 0.99})
pnml_exporter.export_net(net, im, "discovered_pn.pnml")

In [ ]:
# load the heuristics miner petrinet from Q1
net, initial_marking, final_marking = pnml_importer.import_net("discovered_pn.pnml")

In [ ]:
# enhance the log with decay functions
log_wrapper = LogWrapper(event_log, resources=["Categorized_Activity"])
enhanced_pn = EnhancedPN(net, initial_marking)
enhanced_pn.enhance(log_wrapper)
enhanced_pn.saveToFile("enhanced_discovered_pn.json")

In [ ]:
# open enhanced petri net
enhanced_pn = EnhancedPN(net, initial_marking, decay_function_file="enhanced_discovered_pn.json")
tss_json, tss_objs = enhanced_pn.decay_replay(log_wrapper=log_wrapper)

In [ ]:
dir(enhanced_pn.net)

In [ ]:
with open("timedstatesamples.json", 'w') as fp:
    json.dump(tss_json, fp)

In [ ]:
algo = NAPr(tss_train_file="timedstatesamples.json", tss_test_file="timedstatesamples.json", options={"n_epochs" : 10})

In [ ]:
algo.train(checkpoint_path="model-path", name="MODEL-NAME", save_results=True)

In [ ]:
# load the model
algo.loadModel(path="model-path", name="MODEL-NAME")

In [ ]:
# predict values
nap_out, string_out = algo.predict(tss_objs)

In [ ]:
tss_objs[4].data.items()
print(dir(tss_objs[4]))

In [ ]:
# visualize petri net
gviz = visualizer.apply(enhanced_pn.net, initial_marking, final_marking, parameters={visualizer.Variants.WO_DECORATION.value.Parameters.DEBUG: True})
visualizer.view(gviz)

In [ ]:
transitions = net.transitions
# print(len(transitions.difference()))
# print(dir(transitions.__xor__("hid_142")))
enhanced_pn.net.places
# for place in net.places:
#     print("\nPLACE: "+ place.name)
#     for arc in place.in_arcs:
#         print(arc.source.name, arc.source.label)

In [ ]:
tss_loaded_objs[0]

In [ ]:
tss_loaded_objs = loadTimedStateSamples("timedstatesamples.json")
nap_out, string_out = algo.predict([tss_loaded_objs[12]])

In [ ]:
if __name__== "__main__":
    log = xes_import_factory.apply('YOUR_EVENTLOG.xes')

    net, im, fm = heuristics_miner.apply(log, parameters={"dependency_thresh": 0.99})
    pnml_exporter.export_net(net, im, "discovered_pn.pnml")

    net, initial_marking, final_marking = pnml_importer.import_net("discovered_pn.pnml")

    log_wrapper = LogWrapper(log)
    enhanced_pn = EnhancedPN(net, initial_marking)
    enhanced_pn.enhance(log_wrapper)
    enhanced_pn.saveToFile("enhanced_discovered_pn.json")

    enhanced_pn = EnhancedPN(net, initial_marking, decay_function_file="enhanced_discovered_pn.json")
    tss_json, tss_objs = enhanced_pn.decay_replay(log_wrapper=log_wrapper)

    with open("timedstatesamples.json", 'w') as fp:
        json.dump(tss_json, fp)

    algo = NAP(tss_train_file="timedstatesamples.json", tss_test_file="timedstatesamples.json", options={"n_epochs" : 100})
    algo.train(checkpoint_path="model-path", name="MODEL-NAME", save_results=True)

    algo = NAP()
    algo.loadModel(path="model-path", name="MODEL-NAME")

    nap_out, string_out = algo.predict(tss_objs)

    tss_loaded_objs = loadTimedStateSamples("timedstatesamples.json")
    nap_out, string_out = algo.predict(tss_loaded_objs)

## Decision tree

In [ ]:
from pm4py.algo.enhancement.decision import algorithm as decision_mining
X, y, class_names = decision_mining.apply(event_log, enhanced_pn.net, im, fm, decision_point="discard")

In [ ]:

clf, feature_names, classes = decision_mining.get_decision_tree(df_1, net, im, fm, decision_point="pre_calculate")

In [ ]:
# create the mapping
mapping = create_activity_categories(df_1)

# append column with new mapping to our old dataframe
add_mapping_to_df(df_1)

#output first 5 results
df_1.tail()

In [ ]:
csv = df_1.to_csv("df_1.csv")

#event_stream = csv_importer.import_event_stream(os.path.join("tests", "input_data", "running-example.csv"))
log_csv = pd.read_csv('df_1.csv', sep=',')
log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
log_csv = log_csv.sort_values('time:timestamp')
event_log = log_converter.apply(log_csv)

# automatic feature selection on the event log
"""Results in 10 features that will be one hot encoded and used to traint the model"""
""" 8 out of 21 categories were considered important, together with index (so not timestamp) which is because
the traces were already sorted on timestamp. Lastly the Unnamed 0 feature from the event log. 

"""
# data, feature_names = get_log_representation.get_default_representation(event_log)
# feature_names

In [ ]:
event_log[0][0]

In [ ]:
info_dict = {}
for trace in event_log[0]:
    print(trace)

In [ ]:
{'Unnamed: 0': 0, 'index': 25616, 
 'Activity': 'Payment application-Application-mail income', 
 'time:timestamp': Timestamp('2015-04-15 00:00:00+0000', tz='UTC'), 
 'Categorized_Activity': 'Start_Payment', 'doctype': 'Payment application', 'concept:name': 'mail income'}

In [ ]:
totals = []
for trace in event_log:
    for event in trace:
        totals.append(event)

In [ ]:
len(totals)

In [ ]:
data[0]

In [ ]:
event_log

In [ ]:
# Save the one-hot encoding for use in the model
def convert_ndarray_to_list(array, path='model-path/MODEL-NAME_nap_onehotdict.json'):
    lists = array.tolist()
    Path(path).touch(exist_ok=True)
    with open(path, "w") as file:
        json.dump(lists, file)
        
convert_ndarray_to_list(data)

In [ ]:
event_log[0][1]

In [ ]:
data[6]

In [ ]:
def add_next_event(self, df):
        unique_case_ids = df["Case ID"].unique()
        for case in unique_case_ids:

df.loc[df["Case ID"] == "8b99873a6136cfa6"]

In [ ]:
net, im, fm = heuristics_miner.apply(event_log, parameters={"dependency_thresh": 0.99})
pnml_exporter.export_net(net, im, "discovered_pn.pnml")

In [ ]:
# load the heuristics miner petrinet from Q1
net, initial_marking, final_marking = pnml_importer.import_net("discovered_pn.pnml")

In [ ]:
# enhance the log with decay functions
log_wrapper = LogWrapper(event_log, resources=["Categorized_Activity"])
enhanced_pn = EnhancedPN(net, initial_marking)
enhanced_pn.enhance(log_wrapper)
enhanced_pn.saveToFile("enhanced_discovered_pn.json")

In [ ]:
# open enhanced petri net
enhanced_pn = EnhancedPN(net, initial_marking, decay_function_file="enhanced_discovered_pn.json")
tss_json, tss_objs = enhanced_pn.decay_replay(log_wrapper=log_wrapper)

In [ ]:
dir(enhanced_pn.net)

In [ ]:
with open("timedstatesamples.json", 'w') as fp:
    json.dump(tss_json, fp)

In [ ]:
algo = NAPr(tss_train_file="timedstatesamples.json", tss_test_file="timedstatesamples.json", options={"n_epochs" : 10})

In [ ]:
algo.train(checkpoint_path="model-path", name="MODEL-NAME", save_results=True)

In [ ]:
# load the model
algo.loadModel(path="model-path", name="MODEL-NAME")

In [ ]:
# predict values
nap_out, string_out = algo.predict(tss_objs)

In [ ]:
tss_objs[4].data.items()
print(dir(tss_objs[4]))

In [ ]:
# visualize petri net
gviz = visualizer.apply(enhanced_pn.net, initial_marking, final_marking, parameters={visualizer.Variants.WO_DECORATION.value.Parameters.DEBUG: True})
visualizer.view(gviz)

In [ ]:
transitions = net.transitions
# print(len(transitions.difference()))
# print(dir(transitions.__xor__("hid_142")))
enhanced_pn.net.places
# for place in net.places:
#     print("\nPLACE: "+ place.name)
#     for arc in place.in_arcs:
#         print(arc.source.name, arc.source.label)

In [ ]:
tss_loaded_objs[0]

In [ ]:
tss_loaded_objs = loadTimedStateSamples("timedstatesamples.json")
nap_out, string_out = algo.predict([tss_loaded_objs[12]])

In [ ]:
if __name__== "__main__":
    log = xes_import_factory.apply('YOUR_EVENTLOG.xes')

    net, im, fm = heuristics_miner.apply(log, parameters={"dependency_thresh": 0.99})
    pnml_exporter.export_net(net, im, "discovered_pn.pnml")

    net, initial_marking, final_marking = pnml_importer.import_net("discovered_pn.pnml")

    log_wrapper = LogWrapper(log)
    enhanced_pn = EnhancedPN(net, initial_marking)
    enhanced_pn.enhance(log_wrapper)
    enhanced_pn.saveToFile("enhanced_discovered_pn.json")

    enhanced_pn = EnhancedPN(net, initial_marking, decay_function_file="enhanced_discovered_pn.json")
    tss_json, tss_objs = enhanced_pn.decay_replay(log_wrapper=log_wrapper)

    with open("timedstatesamples.json", 'w') as fp:
        json.dump(tss_json, fp)

    algo = NAP(tss_train_file="timedstatesamples.json", tss_test_file="timedstatesamples.json", options={"n_epochs" : 100})
    algo.train(checkpoint_path="model-path", name="MODEL-NAME", save_results=True)

    algo = NAP()
    algo.loadModel(path="model-path", name="MODEL-NAME")
38543
    nap_out, string_out = algo.predict(tss_objs)

    tss_loaded_objs = loadTimedStateSamples("timedstatesamples.json")
    nap_out, string_out = algo.predict(tss_loaded_objs)

## Decision tree

In [ ]:

clf, feature_names, classes = decision_mining.get_decision_tree(df_1, net, im, fm, decision_point="pre_calculate")